In [ ]:
import caveclient
import pandas as pd
import pcg_skel
import cloudvolume
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import skeleton_plot as skelplot
import skeleton_plot.skel_io as skel_io
import trimesh
import navis

from meshparty import trimesh_io
from standard_transform import minnie_transform_nm
from scipy.spatial import cKDTree
from ast import literal_eval

In [ ]:
from utils import MinnieData

mdata = MinnieData()

l23 = mdata.get_dorkenwald_neurons_l23()

In [ ]:
import caveclient
client=caveclient.CAVEclient("minnie65_public",auth_token='8bb19d9702fb74f6d6d01bfb54b85ba7')
skel_path = "s3://bossdb-open-data/iarpa_microns/minnie/minnie65/skeletons/v661/skeletons/"
resolution_scale=1000

materialization = 1300
#skel_materialization = 661 # version at which skeletons were pre-generated
client.version = materialization

In [ ]:
def filter_coord(exp):
        exp = exp[1:-1]
        entries = exp.split(" ")
        entries_filtered = []
        for e in entries:
            if e:
                entries_filtered.append(e)
        return [int(s) for s in entries_filtered]

In [ ]:


for neuron in l23['pre']:
    synapses = client.materialize.tables.synapse_target_predictions_ssa(post_pt_root_id=neuron).query()
    synapses = synapses[synapses.tag.isin(["spine", "shaft"])]
    synapses.to_csv(f"./synapses/{neuron}_synapses.csv")
    synapses = pd.read_csv(f'./synapses/{neuron}_synapses.csv', converters={'post_pt_position': filter_coord})
    transf = minnie_transform_nm()
    resolution = np.array([4, 4, 40])
    mm = trimesh_io.MeshMeta(cv_path = "precomputed://gs://iarpa_microns/minnie/minnie65/seg_m1300",
                            disk_cache_path='minnie65_meshes',
                            map_gs_to_https=True)

    mesh = mm.mesh(seg_id=neuron, lod=3)
    mesh_obj = trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces)
    data = mesh_obj.vertices.copy()
    mesh_obj.vertices = transf.apply(data)
    navis_mesh = navis.MeshNeuron((mesh_obj.vertices,mesh_obj.faces))
    # Perform skeletonization
    skeleton = navis.skeletonize(navis_mesh)
    skeleton = navis.heal_skeleton(skeleton)
    skeleton.units = '1 micron'
    skeleton = navis.resample_skeleton(skeleton, 0.1)
    # Extract skeleton node coordinates.
    # (Adjust the attribute name based on your skeleton object's structure.)
    incoming_synapses = client.materialize.query_table("synapses_pni_2", filter_equal_dict={"post_pt_root_id": neuron})
    outgoing_synapses = client.materialize.query_table("synapses_pni_2", filter_equal_dict={"pre_pt_root_id": neuron})
    incoming_synapses = transf.apply(np.array(incoming_synapses['ctr_pt_position'].values.tolist()) * resolution)
    outgoing_synapses = transf.apply(np.array(outgoing_synapses['ctr_pt_position'].values.tolist()) * resolution)
    # Assume skel.vertices is an (N,3) array and skel.edges is an (M,2) array of vertex‑indices
    verts = np.asarray(skeleton.vertices)         # shape (N,3)
    edges = np.asarray(skeleton.edges)            # shape (M,2)
        # Extract skeleton node coordinates.
        
    skeleton_nodes = skeleton.vertices  # shape: (N, 3)
    tree = cKDTree(skeleton_nodes)
    distances, post_indices = tree.query(incoming_synapses)
    post_coord = skeleton.nodes[['x', 'y', 'z']].values[post_indices]
    post_indices = skeleton.nodes['node_id'].values[post_indices]
    # Outgoing synapses 
    distances, pre_indices = tree.query(outgoing_synapses)
    pre_coord = skeleton.nodes[['x', 'y', 'z']].values[pre_indices]
    pre_indices = skeleton.nodes['node_id'].values[pre_indices]

    # Create a DataFrame for pre-synaptic connectors
    df_pre = pd.DataFrame({
            "connector_id": [f"pre_{i}" for i in range(len(pre_coord))],
            "type": "pre",
            "x": pre_coord[:, 0],
            "y": pre_coord[:, 1],
            "z": pre_coord[:, 2],
            "keys": pre_indices,       # The mapped skeleton node IDs from pre_synapses
            "node_id": pre_indices     # Duplicate column for clarity as requested
        })

    # Create a DataFrame for post-synaptic connectors
    df_post = pd.DataFrame({
            "connector_id": [f"post_{i}" for i in range(len(post_coord))],
            "type": "post",
            "x": post_coord[:, 0],
            "y": post_coord[:, 1],
            "z": post_coord[:, 2],
            "keys": post_indices,      # The mapped skeleton node IDs from post_synapses
            "node_id": post_indices    # Duplicate column for clarity as requested
        })


    # Combine pre- and post-synaptic connectors into one DataFrame.
    df_connectors = pd.concat([df_pre, df_post], ignore_index=True)
    skeleton.connectors = df_connectors
    skel = navis.heal_skeleton(skeleton)
    split =navis.split_axon_dendrite(skel, flow_thresh=0.2)
    for n in split:
        if n.compartment == 'dendrite':
            dendrite = n
        elif n.compartment == 'axon':
            axon = n
    navis.write_swc(axon, f"./skeletons/axons/{neuron}.swc")
    navis.write_swc(dendrite, f"./skeletons/dendrites/{neuron}.swc")
    navis.plot3d([axon, dendrite])
    plt.show()
